# **AlTar 2 - a toy model with epistemic uncertainties**

# **Step 2 - run AlTar2**

This notebook follows the `step 1` notebook in which the input files for **AlTar2** have been prepared.

As a reminder, we want to incorporate the epistemic uncertainties in our sampling process. To do so, we have prepared the configuration `test1.pfg` to update $\mathbf{C}_\mathrm{p}$ at every step of sampling procedure.
Here are the input files for our toy model:
- `test1.gf.h5`
- `test1d.h5`
- `test1.Cd.h5`
- `test1.Cov.h5`
- `test1.mprior.h5`
- `test1.KElastic.h5`

The *updated $\mathbf{C}_\mathrm{p}$* option currently only works on GPU. Within this notebook, we only have access to CPU. We will then perform the sampling process with a static $\mathbf{C}_\mathrm{p}$ incorporated as the misfit covariance matrix $\mathbf{C}_\mathrm{\chi}$. In this case, our input files become:
- `test1.gf.h5`
- `test1.d.h5`
- `test1.Cx.h5`  (sum of $\mathbf{C}_\mathrm{d}$ and $\mathbf{C}_\mathrm{p}$)

In this example, we will:
1. Update the configuration file, for use with CPUs
2. Run Altar2
3. Analyze the results

### 1. Update the `test1.pfg` configuration file to `slipmodel.pfg`, for use with CPUs

We modify the configuration file for use with CPUs, and with a static $\mathbf{C}_\mathrm{p}$.

The `slipmodel.pfg` file is:

    ;
    ; michael a.g. aïvázis
    ; orthologue
    ; (c) 1998-2019 all rights reserved
    ;

    ; the application
    ; the name should be the same as the AlTar application instance (name='linear')
    linear:
        ; the model configuration
        model:
            case = test1
            ; the number of free parameters
            parameters = 20
            ; the number of observations
            observations = 300

            ; data observations
            data = test1.d.h5

            ; data covariance/uncertainties
            cd = test1.Cx.h5

            ; Green's function 
            green = test1.gf.h5

            ; prior distribution for parameters
            prep = altar.models.seismic.moment
            prep:
                support = (0., 20.) ; slip range
                Mw_mean = 8.08
                Mw_sigma = 0.3
                Mu = [30] ; in GPa
                area = [250000000.0] ; patch area in m2

            prior:
                parameters = {linear.model.parameters}
                center = 10.
                sigma = 3.

        ; run configuration
        job.tasks = 1 ; number of tasks per host
        job.gpus = 0  ; number of gpus per task
        job.chains = 2**10 ; number of chains per task


    ; end of file

### 2. Run **AlTar**

We follow the same steps as in the tutorial *An introduction to Altar2 framework - from a Linear Model example*. We will not detail the steps, so please refer to this tutorial for details.

In [1]:
import altar
import altar.models.linear

# make a specialized app that uses the linear model by default
class LinearApp(altar.shells.altar, family='altar.applications.linear'):
    """
    A specialized AlTar application that exercises the Linear model
    """
    # user configurable component
    model = altar.models.model(default='linear')

# create an instance
myapp = LinearApp(name='linear')
# call initialize method to prepare its compone
myapp.initialize()
# to sample the posterior with CATMIP alogorithm
myapp.model.posterior(application=myapp)

ModuleNotFoundError: No module named 'altar'

In [2]:
# to obtain the final step data
step=myapp.controller.worker.step
print('beta =', step.beta)
print('samples =', step.samples)
print('parameters =', step.parameters)
# to obtain theta matrix dim(samples, parameters)
theta = step.theta
mean, sd = theta.mean_sd(axis=0) # axis=0 for averaging over samples

model = myapp.model
data = myapp.data

NameError: name 'myapp' is not defined

### 3. Analyze the results

#### 3.1 Represent the inferred slip

We first write a function to represent the mean and standard deviations of the inferred slip models.

In [8]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## Create the colormap
color1 = [(218,240,178),(163,219,184),(96,194,192),(47,163,194),(32,120,180),(36,73,158)]
color12 = [(218,240,178),(163,219,184),(96,194,192),(47,163,194),(32,120,180)]
color2 = [(253,208,162),(253,174,107),(253,141,60),(241,105,19),(217,72,1),(166,54,3)]
colornames = list(reversed(color2)) + color1 + list(reversed(color12)) + color2

## Function to plot slip
def plotSlipBox(moy, std, ns, nd, bounds, **params):
    '''
    Plot every subfault as a gradient between mean - std and mean + std

    IN ARGUMENT:
    You need to specify at least 8 arguments:
     --> mean
     --> standard deviation
     --> ns = number of patches in strike
     --> nd = number of patches in dip
     --> bounds = boudaries for slip
    '''
        
    # define slip steps
    x00 = np.linspace(bounds[0],bounds[1],49)

    plt.figure(figsize=((ns * 5 / 2) / 1.2, ((nd + 1) * 53.5 / 200) / 1.5))
    # colorbar
    cmape = colors.LinearSegmentedColormap.from_list('cptslip', [[i / 255 for i in x] for x in colornames])
    cmape.set_over('#a63603')
    cmape.set_under('#a63603')
    cNorm = colors.Normalize(vmin=bounds[0], vmax=bounds[1])
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmape)
    scalarMap.set_array([])

    # Loop over subfaults
    c = 1
    l = 0
    for i in range(Np // 2):
        if i % ns == 0:
            # j=ns-1
            l = l + 1
            c = 1
        a = plt.subplot2grid((nd + 2, ns + 1), (l, c), colspan=1, rowspan=1)
        # j=j-1
        c = c + 1
        # plot gradient from mean-std to mean
        plt.imshow([np.linspace(moy[i] - std[i] - 0.1, moy[i], 30)],
                   cmap=cmape,
                   interpolation='bicubic',
                   extent=[moy[i] - std[i], moy[i], 0, 3],
                   vmin=boundss[0], vmax=boundss[1],
                   alpha = 0.75,zorder=3)
        # plot gradient from mean to mean+std
        plt.imshow([np.linspace(moy[i], moy[i] + std[i] + 0.1, 30)],
                   cmap=cmape,
                   interpolation='bicubic',
                   extent=[moy[i], moy[i]+std[i], 0, 3],
                   vmin=boundss[0], vmax=boundss[1],
                   alpha = 0.75,zorder=4)
        # plot the mean value
        colorval = scalarMap.to_rgba(abs(moy[i]))
        rectangle = plt.Rectangle((moy[i]-0.5,-0.5), 1.0, 4, fc=colorval,ec="white",lw=0.5,zorder=5)
        plt.gca().add_patch(rectangle)
        # plot an horizontal line
        plt.axhline(y=1.5,linewidth=0.1, color=darklavend,zorder=2)
        # add a vertical line to represent the target model
        plt.axvline(x=10.,linewidth=0.2, color=darkblue,alpha=0.5,zorder=2)
        # ticks aspect
        plt.box(False)
        a.tick_params(axis="both", which="both", bottom="off", top="off",
                      labelbottom="off", left="off", right="off", labelleft="off")
        plt.xlim(min(x00)-0.1,max(x00)+0.1)
        plt.ylim(-0.5,3.5)
    plt.tight_layout(pad=0.05)
    plt.show()
    return

In [6]:
ns = 1
nd = 20
bounds = [4.,16.]
plotSlipBox(mean, sd, ns, nd, bounds)

NameError: name 'mean' is not defined

#### 3.2 Represent the fit of the predictions to the observations

In [ ]:
pred = model.green * mean
pred_sd = model.green * sd

In [7]:
x = np.linspace(-40000,40000,100)

fig=plt.figure(figsize=(7,2))
plt.axvline(x=0,color='lightgray')
plt.fill_between(x/1000, data - data_sd, data + data_sd,color=orange, alpha=0.1)
plt.fill_between(x/1000, pred - pred_sd, pred + pred_sd, color=blue, alpha=0.1)
plt.plot(x/1000, data, color=orange, label='Target predictions')
plt.plot(x/1000, pred, '--',color=blue, label='Predictions with Cp')    
plt.xlim([-35,35])
plt.xlabel('Distance perpendicular to fault (km)')
plt.ylabel('Surface displacement (m)')
plt.show()
fig.savefig('surface_disp_with_cp.pdf',format='pdf')

NameError: name 'np' is not defined